In [1]:
import pandas as pd
import uuid
from dataframe_sql import register_temp_table, query
import ast
import random
from tqdm import tqdm

In [2]:
test_set = pd.read_json('wikisql_ann_test.json')

In [3]:
test_set.head()

,answer,question,id,header,rows,header_types
0,SELECT ` <col2> ` FROM ` table ` WHERE ` <col0...,<col0> Player : text <col1> No. : text <col2> ...,4e84108c351d47b5aedac962af6749cf,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
1,SELECT ` <col5> ` FROM ` table ` WHERE ` <col4...,<col0> Player : text <col1> No. : text <col2> ...,f6e1c2213a1d4cebbb1521da219c7aa0,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
2,SELECT ` <col5> ` FROM ` table ` WHERE ` <col4...,<col0> Player : text <col1> No. : text <col2> ...,1ce1252d4098425aaa9c38aeb5b79c69,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
3,SELECT COUNT ` <col5> ` FROM ` table ` WHERE `...,<col0> Player : text <col1> No. : text <col2> ...,33caff92db0442fa8870dee4ace70c4a,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
4,SELECT ` <col2> ` FROM ` table ` WHERE ` <col3...,<col0> No : real <col1> Date : text <col2> Rou...,acaa5428b2d541c7a7842de42e6fc77f,"['No', 'Date', 'Round', 'Circuit', 'Pole Posit...","[['1', '24 February', 'Qatar', 'Losail', 'Kevi...","['real', 'text', 'text', 'text', 'text', 'text..."


In [4]:
def register_table(query, data, row_id):
    tab_id = random.randint(0, 10000000)
    query = query.replace('` table `', f"table_{tab_id}")
    query = query.replace('` ','\'').replace(' `','\'')
    register_temp_table(data, f"table_{tab_id}")
    return query

In [7]:
answers = []
for idx, row in tqdm(test_set.iterrows()):
    answer = [[]]
    try:
        sql = row.answer
        header = ast.literal_eval(row.header)
        header = [col.replace(" ", "_").replace('/','_') for col in header]
        rows = ast.literal_eval(row.rows)
        table = pd.DataFrame(rows, columns=header)
        for ix, column in enumerate(header):
            if f"` <col{ix}> `" in sql:
                if ' ' in column:
                    sql = sql.replace(f"` <col{ix}> `", f'\'{column}\'')
                else:
                    sql = sql.replace(f"` <col{ix}> `", column)
        answers.append(query(register_table(query=sql, data=table, row_id=idx)).values)
    except:
        answers.append(answer)
        continue

15878it [39:48,  6.65it/s]


In [9]:
test_set['answer_rows'] = answers

In [11]:
test_set.to_csv('wikisql_ann_test_tapas.csv', index=False)